<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown 165LwqivtdzeXwMaj2VeGzgspqdnOiyrq

Downloading...
From (original): https://drive.google.com/uc?id=165LwqivtdzeXwMaj2VeGzgspqdnOiyrq
From (redirected): https://drive.google.com/uc?id=165LwqivtdzeXwMaj2VeGzgspqdnOiyrq&confirm=t&uuid=a2995972-fbc4-4492-8976-e5ab7d1aa4d9
To: /content/Shark_project_split.zip
100% 139M/139M [00:00<00:00, 200MB/s]


In [6]:
# -------------------------------------------
# 1. فك الضغط وتثبيت المتطلبات
# -------------------------------------------
print("🔧 المرحلة 1: فك الضغط وتثبيت المكتبات...")
#!unzip -q "/content/Shark_project_split.zip" -d "/content/Shark_project_split"


!pip install -q transformers datasets torchvision evaluate wandb

# -------------------------------------------
# 2. تحميل البيانات باستخدام ImageFolder (من المسار الأصلي)
# -------------------------------------------
print("📁 المرحلة 2: تحميل الصور وتحضير التحويلات...")
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os

data_dir = "/content/Shark_project_split"  # التزامًا بمسارك

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

dataset = ImageFolder(data_dir, transform=transform)
print("✅ عدد الصور:", len(dataset))
print("✅ الفئات:", dataset.classes)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
#print("📊 تقسيم البيانات: تدريب =", len(train_dataset), " | تحقق =", len(val_size))

print("📊 تقسيم البيانات: تدريب =", len(train_dataset), " | تحقق =", val_size)

# -------------------------------------------
# 3. تحويل البيانات إلى Hugging Face Dataset
# -------------------------------------------
print("🔄 المرحلة 3: تحويل البيانات إلى Dataset من مكتبة HuggingFace...")
from datasets import Dataset as HFDataset

def convert_to_hf_dataset(torch_dataset):
    images = []
    labels = []
    for img, label in torch_dataset:
        pil_img = to_pil_image(img)
        images.append(pil_img)
        labels.append(label)
    return HFDataset.from_dict({"image": images, "label": labels})

hf_train = convert_to_hf_dataset(train_dataset)
hf_val = convert_to_hf_dataset(val_dataset)
print("✅ تم التحويل إلى HFDataset")

# -------------------------------------------
# 4. تحميل النموذج و feature extractor
# -------------------------------------------
print("📦 المرحلة 4: تحميل نموذج ViT و ViTImageProcessor...")
from transformers import ViTForImageClassification, ViTImageProcessor

feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(dataset.classes),
    id2label={str(i): c for i, c in enumerate(dataset.classes)},
    label2id={c: str(i) for i, c in enumerate(dataset.classes)},
    ignore_mismatched_sizes=True
)
print("✅ النموذج جاهز للتدريب")

# -------------------------------------------
# 5. تطبيق الـ transform على HuggingFace Dataset
# -------------------------------------------
print("🧪 المرحلة 5: تجهيز البيانات بالـ feature extractor...")

def transform_example(example):
    encoding = feature_extractor(images=example['image'], return_tensors="pt")
    encoding['label'] = example['label']
    return {k: v.squeeze() for k, v in encoding.items()}

hf_train = hf_train.with_transform(transform_example)
hf_val = hf_val.with_transform(transform_example)
print("✅ تم تجهيز البيانات للتحويل")

# -------------------------------------------
# 6. إعداد التدريب
# -------------------------------------------
print("⚙️ المرحلة 6: إعداد تدريب النموذج...")
from transformers import TrainingArguments, Trainer
from evaluate import load
import numpy as np

os.environ["WANDB_DISABLED"] = "true"
accuracy = load("accuracy")

training_args = TrainingArguments(
    output_dir="./vit-shark-classifier",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    compute_metrics=compute_metrics
)

# -------------------------------------------
# 7. بدء التدريب
# -------------------------------------------
print("🚀 المرحلة 7: بدء التدريب...")
trainer.train()
print("✅ التدريب انتهى")

# -------------------------------------------
# 8. حفظ النموذج و feature extractor
# -------------------------------------------
print("💾 المرحلة 8: حفظ النموذج والـ processor...")
save_path = "/content/vit-shark-final"
model.save_pretrained(save_path)
feature_extractor.save_pretrained(save_path)
print(f"✅ النموذج محفوظ في: {save_path}")

🔧 المرحلة 1: فك الضغط وتثبيت المكتبات...
📁 المرحلة 2: تحميل الصور وتحضير التحويلات...
✅ عدد الصور: 493
✅ الفئات: ['test', 'train', 'val']
📊 تقسيم البيانات: تدريب = 394  | تحقق = 99
🔄 المرحلة 3: تحويل البيانات إلى Dataset من مكتبة HuggingFace...
✅ تم التحويل إلى HFDataset
📦 المرحلة 4: تحميل نموذج ViT و ViTImageProcessor...


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ النموذج جاهز للتدريب
🧪 المرحلة 5: تجهيز البيانات بالـ feature extractor...
✅ تم تجهيز البيانات للتحويل
⚙️ المرحلة 6: إعداد تدريب النموذج...
🚀 المرحلة 7: بدء التدريب...


KeyError: 'image'

In [ ]:
print(dataset['train'][0])

In [8]:
# افترض أن train_dataset هو ناتج random_split
print(train_dataset[0])  # هيطبع أول عنصر: (الصورة، الليبل)

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 0)


In [11]:

import os
import shutil
from PIL import Image
import torchvision.transforms as T
import torch

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

def is_valid_image(path):
    try:
        image = Image.open(path).convert("RGB")
        tensor = transform(image)
        if tensor.sum() == 0:
            print(f"[⚠️ صفرية] {path}")
            return False
        return True
    except Exception as e:
        print(f"[❌ خطأ] {path} - {e}")
        return False

def get_valid_image_paths(directory, bad_folder="bad_images"):
    valid_paths = []
    total = 0
    bad = 0

    bad_folder_path = os.path.join(directory, bad_folder)
    os.makedirs(bad_folder_path, exist_ok=True)

    for root, _, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(('.jpg', '.jpeg', '.png')):
                total += 1
                path = os.path.join(root, name)
                if is_valid_image(path):
                    valid_paths.append(path)
                else:
                    bad += 1
                    new_path = os.path.join(bad_folder_path, os.path.relpath(path, directory))
                    os.makedirs(os.path.dirname(new_path), exist_ok=True)
                    shutil.move(path, new_path)
                    print(f"👉 نُقل إلى: {new_path}")

    print(f"\n📊 الإحصائيات:")
    print(f"✅ عدد الصور الصالحة: {len(valid_paths)}")
    print(f"❌ عدد الصور الغير صالحة: {bad}")
    print(f"📦 إجمالي عدد الصور: {total}")

    return valid_paths

valid_image_paths = get_valid_image_paths("/content/Shark_project_split")


📊 الإحصائيات:
✅ عدد الصور الصالحة: 493
❌ عدد الصور الغير صالحة: 0
📦 إجمالي عدد الصور: 493


In [ ]:
print(dataset['train'][0])

In [ ]:
for example in dataset['train']:
    print(example.keys())
    break

In [14]:
img, label = train_data[0]
print(type(img))      # حيكون <class 'PIL.Image.Image'>
print(label)          # رقم الصنف

NameError: name 'train_data' is not defined

In [21]:
from datasets import load_dataset

data_path = "/content/Shark_project_split"
print("📂 تحميل الصور...")
dataset = load_dataset("imagefolder", data_dir=data_path)
print("✅ تم تحميل البيانات")
print(dataset)

📂 تحميل الصور...


ValueError: Invalid pattern: '**' can only be an entire path component

In [24]:
!ls /content/Shark_project_split/train
# يجب أن يعطيك: Blacktip_Shark/ Bull_Shark/ ... (مجلدات كل نوع)

Blacktip_Shark	Great_White_Shark  Mako_Shark	Whale_Shark
Bull_Shark	Hammerhead_Shark   Tiger_Shark	Whitetip_Shark


In [25]:

import os
for split in ['train', 'val', 'test']:
    print(f"📁 محتويات {split}:")
    print(os.listdir(f"/content/Shark_project_split/{split}"))

📁 محتويات train:
['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']
📁 محتويات val:
['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']
📁 محتويات test:
['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']


In [2]:

from datasets import load_dataset, DatasetDict
from transformers import ViTForImageClassification, ViTFeatureExtractor, TrainingArguments, Trainer
import torch
import numpy as np
import os

# ✅ تأكد من إن المسار صحيح
data_dir = "/content/Shark_project_split"

# ✅ تحميل البيانات من الفولدرات الثلاثة
dataset = DatasetDict({
    "train": load_dataset("imagefolder", data_dir=f"{data_dir}/train")["train"],
    "val": load_dataset("imagefolder", data_dir=f"{data_dir}/val")["train"],
    "test": load_dataset("imagefolder", data_dir=f"{data_dir}/test")["train"],
})

# ✅ تعريف المعالج (feature extractor)
extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

# ✅ عدد الأصناف تلقائيًا
labels = dataset["train"].features["label"].names
num_labels = len(labels)

# ✅ تحويل الصور
def transform(example):
    return extractor(images=example['image'], return_tensors="pt")

dataset = dataset.with_transform(transform)

# ✅ تحميل النموذج
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_labels,
    id2label={str(i): l for i, l in enumerate(labels)},
    label2id={l: str(i) for i, l in enumerate(labels)},
)

# ✅ دالة دقة التقييم
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

# ✅ إعدادات التدريب
training_args = TrainingArguments(
    output_dir="./vit-shark-output",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
)

# ✅ إعداد المدرب
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    compute_metrics=compute_metrics,
    tokenizer=extractor,
)

# ✅ بدء التدريب
trainer.train()

# ✅ اختبار النموذج على مجموعة الاختبار
trainer.evaluate(dataset["test"])

ValueError: Invalid pattern: '**' can only be an entire path component

In [3]:
print(f"{data_dir}/train")

/content/Shark_project_split/train


In [5]:
import os

print("Train:", os.listdir("/content/Shark_project_split/train")[:8])
print("Val:", os.listdir("/content/Shark_project_split/val")[:8])
print("Test:", os.listdir("/content/Shark_project_split/test")[:8])

Train: ['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']
Val: ['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']
Test: ['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']


In [3]:
from datasets import DatasetDict, load_dataset

data_dir = "/content/Shark_project_split"

try:
    # ✅ تحميل البيانات من الفولدرات الثلاثة

    dataset = DatasetDict({
        "train": load_dataset("imagefolder", data_dir=f"{data_dir}/train")["train"],
       "val": load_dataset("imagefolder", data_dir=f"{data_dir}/val")["train"],
       "test": load_dataset("imagefolder", data_dir=f"{data_dir}/test")["train"],
    })

    # ✅ عرض بعض العينات
    print("📊 حجم كل جزء:")
    for split in dataset:
        print(f"{split}: {len(dataset[split])} صورة")

    print("\n🎯 عينة من train:")
    print(dataset["train"][0])

except Exception as e:
    print("❌ حصل خطأ:")
    print(e)

❌ حصل خطأ:
Invalid pattern: '**' can only be an entire path component


In [4]:

from datasets import DatasetDict, load_dataset

data_dir = "/content/Shark_project_split" # This is your root directory

try:
    # ✅ تحميل البيانات من الفولدرات الثلاثة
    # Point load_dataset to the root directory
    # It will automatically look for 'train', 'validation', 'test' subdirectories
    # and load them as separate splits.
    dataset = load_dataset("imagefolder", data_dir=data_dir)

    # If your splits are named differently (e.g., "validation" instead of "val")
    # or you only have a "train" folder, the result of load_dataset will
    # be a DatasetDict containing only the detected splits.

    # ✅ عرض بعض العينات
    print("📊 حجم كل جزء:")
    for split in dataset:
        print(f"{split}: {len(dataset[split])} صورة")

    # Assuming 'train' split exists after loading
    if "train" in dataset:
        print("\n🎯 عينة من train:")
        print(dataset["train"][0])
    else:
        print("\n⚠️ لا يوجد قسم 'train' في مجموعة البيانات المحملة.")

except Exception as e:
    print("❌ حصل خطأ:")
    print(e)

❌ حصل خطأ:
Invalid pattern: '**' can only be an entire path component


In [18]:
from datasets import load_dataset

data_dir = "/content/Shark_project_split"

# تحميل فقط جزء التدريب
dataset = load_dataset("imagefolder", data_dir=f"{data_dir}/train")

# عرض أول عينة
print(dataset["train"][0])

ValueError: Invalid pattern: '**' can only be an entire path component

In [20]:

from datasets import load_dataset

data_dir = "/content/Shark_project_split"

dataset = load_dataset("imagefolder", data_dir=f"{data_dir}/train")
dataset

ValueError: Invalid pattern: '**' can only be an entire path component

In [1]:
from datasets import load_dataset

data_dir = "/content/Shark_project_split"

# Point to the main directory that contains 'train', 'validation', 'test' subfolders
dataset = load_dataset("imagefolder", data_dir=data_dir)
dataset

ValueError: Invalid pattern: '**' can only be an entire path component

In [13]:

import os

data_dir = "/content/Shark_project_split/train"
print(os.listdir(data_dir))  # لازم يطبع مجلدات الكلاسات

class_dir = os.path.join(data_dir, os.listdir(data_dir)[0])
print(os.listdir(class_dir))  # لازم يطبع أسماء الصور

['Mako_Shark', 'Bull_Shark', 'Tiger_Shark', 'Whale_Shark', 'Hammerhead_Shark', 'Great_White_Shark', 'Blacktip_Shark', 'Whitetip_Shark']
['Mako_Shark_0012.jpeg', 'Mako_Shark_0003.jpeg', 'Mako_Shark_0008.jpeg', 'Mako_Shark_0006.jpeg', 'Mako_Shark_0000.jpeg', 'Mako_Shark_0004.jpeg', 'Mako_Shark_0005.jpeg', 'Mako_Shark_0007.jpeg', 'Mako_Shark_0001.jpeg', 'Mako_Shark_0002.jpeg', 'Mako_Shark_0010.jpeg', 'Mako_Shark_0011.jpeg', 'Mako_Shark_0009.jpeg']


In [14]:
import os

data_dir = "/content/Shark_project_split/train"

for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_path):
        for fname in os.listdir(class_path):
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                print("❌ Not an image:", fname)
            elif len(fname) > 100:
                print("⚠️ Name too long:", fname)
            elif not fname.isascii():
                print("⚠️ Non-ASCII name:", fname)

In [15]:
import os

def rename_all_images_uniform(base_dirs):
    index = 0
    for root_dir in base_dirs:
        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for filename in sorted(os.listdir(class_path)):
                    old_path = os.path.join(class_path, filename)
                    ext = os.path.splitext(filename)[1].lower()
                    new_filename = f"image{index}{ext}"
                    new_path = os.path.join(class_path, new_filename)
                    os.rename(old_path, new_path)
                    index += 1

# حدد مجلدات التقسيم
base_dirs = [
    "/content/Shark_project_split/train",
    "/content/Shark_project_split/val",
    "/content/Shark_project_split/test"
]

rename_all_images_uniform(base_dirs)